In [ ]:
!apt-get install flex
!apt-get install bison

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libfl-dev libfl2
Suggested packages:
  bison flex-doc
The following NEW packages will be installed:
  flex libfl-dev libfl2
0 upgraded, 3 newly installed, 0 to remove and 15 not upgraded.
Need to get 334 kB of archives.
After this operation, 1,127 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 flex amd64 2.6.4-6.2 [317 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libfl2 amd64 2.6.4-6.2 [11.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/main amd64 libfl-dev amd64 2.6.4-6.2 [6,316 B]
Fetched 334 kB in 0s (906 kB/s)
Selecting previously unselected package flex.
(Reading database ... 123105 files and directories currently installed.)
Preparing to unpack .../flex_2.6.4-6.2_amd64.deb ...
Unpacking flex (2.6.4-6.2) ...
Selecting previously unselected package libfl2:amd64.
Pre

## Librerias

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### LEX

In [4]:
%%writefile ejercicio1.l
%{
  #include<stdio.h>
  #include "y.tab.h"
%}

%option noyywrap

%%
if                           {return(IF);}
else                         {return(ELSE);}
[A-Za-z]([A-Aa-z]|[0-9])*    {return(IDENTI);}
[0-9]+                       {return(NUMERO);}
"=="                         {return(EQUALS);}
"!="                         {return(NOTEQUALS);}
"<"                          {return(LT);}
">"                          {return(GT);}
"<="                         {return(LEQ);}
">="                         {return(GEQ);}
"NOT"                        {return(NOT);}
"AND"                        {return(AND);}
"OR"                         {return(OR);}
[\t]                         {;}
\n                           {yyterminate();}
.                            { return yytext[0];}
%%

Overwriting ejercicio1.l


### YACC

In [53]:
%%writefile ejercicio1.y
%{

int yylex(void);
FILE *yyin;


char pila[100][10];
int top=0;
char temp[3]="t0";
char labels[3]="L0";

void yyerror (char const *s)
{
  fprintf (stderr, "%s\n", s);
}


void push()
{
  strcpy(pila[++top],yytext);
}


void generate_code()
{
  printf("%s = %s %s %s \n", temp, pila[top-2],pila[top-1],pila[top]);
  top-=2;
  strcpy(pila[top],temp);
  temp[1]++;
}

void generate_assign()
{
  printf("%s = %s\n",pila[top-2],pila[top]);
  top-=2;
}


void generate_boolean()
{
  printf("%s = ( %s %s %s ) \n", temp, pila[top-2],pila[top-1],pila[top]);
  top-=2;
  strcpy(pila[top],temp);
  temp[1]++;
}

void codegen_not()
{
  printf("%s = not %s\n",temp,pila[top]);
  top-=2;
  strcpy(pila[top],temp);
  temp[1]++;
}


void L()
{
  labels[1]--;
  printf("%s\n",labels);
}


void Lcond()
{
  printf("%s = not %s \n",temp,pila[top]);
  printf("if %s goto %s\n",temp,labels);
  temp[1]++;

}

void Lend()
{
  labels[1]++;
  printf("goto %s\n",labels);
}

void Label()
{
  printf("%s\n",labels);
}

void Label_next()
{
  labels[1]++;
  printf("%s\n",labels);
}


void End()
{
  printf("End\n");
}

%}

%token IDENTI NUMERO IF ELSE TRUE FALSE AND OR
%token EQUALS NOTEQUALS LT GT LEQ GEQ NOT
%right '='
%nonassoc OR
%nonassoc AND
%nonassoc EQUALS NOTEQUALS LT GT LEQ GEQ
%nonassoc NOT
%left '+' '-'
%left '*' '/'


%%

S : IF_ANIDADO {End();}
;

IF_SENT : IF '(' B ')' {Lcond();} '{' SENT '}'
;

IF_ANIDADO : IF_SENT {Label();} | IF_SENT {Lend();}  ELSE {L();} '{' SENT '}' {Label_next();}
;

SENT : IF_ANIDADO | SENT E ';' | E ';'
;

B : V EQUALS {push();} E {generate_boolean();}
  | V NOTEQUALS {push();} E {generate_boolean();}
  | V LT {push();} E {generate_boolean();}
  | V GT {push();} E {generate_boolean();}
  | V LEQ {push();} E {generate_boolean();}
  | V GEQ {push();} E {generate_boolean();}
  | '(' B ')'
  | NOT {push();} B {codegen_not();}
  | B AND {push();} B {generate_boolean();}
  | B OR {push();} B {generate_boolean();}
;

E : V '='   {push();}  E {generate_assign();}
  | E '+'   {push();}  E {generate_code();}
  | E '-'   {push();}  E {generate_code();}
  | E '*'   {push();}  E {generate_code();}
  | E '/'   {push();}  E {generate_code();}
  | '(' E ')'
  | V
  | NUMERO {push();}
;

V : IDENTI {push();}
;
%%


void main()
{
  printf("Ingresar expresion:");
  yyparse();
}

Overwriting ejercicio1.y


## TESTING

In [57]:
!flex ejercicio1.l
!yacc ejercicio1.y -d
!gcc y.tab.c -ll
!./a.out
# Probar con esto o parecido
# if(x<5){z=1;}else{t=4;}

Ingrese su expresion:if(x!=2){if(x>4){y=3;}}else{w=3;}
t0 = ( x != 2 ) 
t1 = not t0 
if t1 goto L0
t2 = ( x > 4 ) 
t3 = not t2 
if t3 goto L0
y = 3
L0
goto L1
L0
w = 3
L1
End
